In [16]:
#%pip install crewai==0.119.0 crewai-tools==0.44.0 openai==1.25.1 google-cloud-bigquery==3.17.2 google-auth==2.29.0 google-auth-oauthlib==1.2.0 beautifulsoup4==4.12.3 playwright==1.44.0

In [1]:
import warnings

warnings.filterwarnings("ignore")

import getpass
import os
from typing import Optional

from crewai import Agent, Crew, Task
from crewai.tools import BaseTool
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from google.cloud import bigquery
from google.oauth2 import service_account

In [2]:
# Set up credentials
credentials = service_account.Credentials.from_service_account_file(
    "adsp-34002-on02-prep-sense-58b6ff084106.json"
)
bq_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [3]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key:  ········


In [4]:
os.environ["SERPER_API_KEY"] = getpass.getpass("Enter your Serper API key: ")

Enter your Serper API key:  ········


In [5]:
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"

In [6]:
# Initialize external tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [7]:
from pydantic import BaseModel, Field


class BigQueryPantryToolInput(BaseModel):
    user_id: int = Field(..., description="The user ID for whom to fetch pantry items")
    pantry_table: Optional[str] = Field(None, description="The BigQuery pantry table")

In [8]:


class BigQueryPantryTool(BaseTool):
    name: str = "BigQueryPantryTool"
    description: str = "Fetches pantry items from BigQuery"
    args_schema: type[BaseModel] = BigQueryPantryToolInput

    def _run(self, user_id: int, pantry_table: Optional[str] = None) -> list:
        print(f"[DEBUG] Tool invoked with user_id={user_id}, pantry_table={pantry_table}")
        pantry_table = pantry_table or "adsp-34002-on02-prep-sense.Inventory.pantry_items"

        query = """
            SELECT
              p.user_id,
              pi.pantry_item_id,
              p.pantry_name,
              pp.product_name,
              pi.quantity,
              pi.used_quantity,
              pi.unit_of_measurement,
              pi.expiration_date,
              pi.created_at
            FROM `adsp-34002-on02-prep-sense.Inventory.pantry_items` AS pi
            JOIN `adsp-34002-on02-prep-sense.Inventory.pantry` AS p
              ON pi.pantry_id = p.pantry_id
            JOIN `adsp-34002-on02-prep-sense.Inventory.products` AS pp
              ON pi.pantry_item_id = pp.pantry_item_id
            WHERE p.user_id = @user_id
              AND pi.quantity > pi.used_quantity;
        """

        job_config = bigquery.QueryJobConfig(
            query_parameters=[
                bigquery.ScalarQueryParameter("user_id", "INT64", user_id)
            ]
        )

        try:
            query_job = bq_client.query(query, job_config=job_config)
            results = query_job.result()
            return [dict(row) for row in results]
        except Exception as e:
            return [{"error": str(e)}]

In [9]:
# testing for a specific user

bigquery_tool = BigQueryPantryTool()
results = bigquery_tool._run(user_id=102)
print(results)

[DEBUG] Tool invoked with user_id=102, pantry_table=None
[{'user_id': 102, 'pantry_item_id': 10002, 'pantry_name': 'kitchen_stock', 'product_name': 'mixed fruits', 'quantity': 1.0, 'used_quantity': 0, 'unit_of_measurement': 'box', 'expiration_date': datetime.date(2025, 5, 15), 'created_at': datetime.datetime(2025, 5, 7, 18, 4, 34, 965265)}, {'user_id': 102, 'pantry_item_id': 20004, 'pantry_name': 'kitchen_stock', 'product_name': 'rice', 'quantity': 2.0, 'used_quantity': 0, 'unit_of_measurement': 'kg', 'expiration_date': datetime.date(2025, 8, 10), 'created_at': datetime.datetime(2025, 5, 17, 9, 35, 40, 478671)}, {'user_id': 102, 'pantry_item_id': 20001, 'pantry_name': 'kitchen_stock', 'product_name': 'milk', 'quantity': 1.0, 'used_quantity': 0, 'unit_of_measurement': 'liter', 'expiration_date': datetime.date(2025, 8, 15), 'created_at': datetime.datetime(2025, 5, 17, 9, 35, 40, 478671)}, {'user_id': 102, 'pantry_item_id': 20005, 'pantry_name': 'kitchen_stock', 'product_name': 'vinegar',

In [15]:
## Ingredient filter class

bq_client = bigquery.Client()

class IngredientFilterTool(BaseTool):
    name: str = "IngredientFilterTool"
    description: str = "Fetches non-expired and available pantry items for a given user from BigQuery"
    args_schema: type[BaseModel] = BigQueryPantryToolInput

    def _run(self, user_id: int, pantry_table: Optional[str] = None) -> list:
        print(f"[DEBUG] Tool invoked with user_id={user_id}, pantry_table={pantry_table}")
        pantry_table = pantry_table or "adsp-34002-on02-prep-sense.Inventory.pantry_items"

        query = """
            SELECT
              p.user_id,
              pi.pantry_item_id,
              p.pantry_name,
              pp.product_name,
              pi.quantity,
              pi.used_quantity,
              pi.unit_of_measurement,
              pi.expiration_date,
              pi.created_at
            FROM `adsp-34002-on02-prep-sense.Inventory.pantry_items` AS pi
            JOIN `adsp-34002-on02-prep-sense.Inventory.pantry` AS p
              ON pi.pantry_id = p.pantry_id
            JOIN `adsp-34002-on02-prep-sense.Inventory.products` AS pp
              ON pi.pantry_item_id = pp.pantry_item_id
            WHERE p.user_id = @user_id
              AND pi.quantity > pi.used_quantity
              AND pi.expiration_date >= CURRENT_DATE()
        """

        job_config = bigquery.QueryJobConfig(
            query_parameters=[
                bigquery.ScalarQueryParameter("user_id", "INT64", user_id)
            ]
        )

        try:
            query_job = bq_client.query(query, job_config=job_config)
            results = query_job.result()
            return [dict(row) for row in results]
        except Exception as e:
            return [{"error": str(e)}]

In [17]:
ingredient_tool = IngredientFilterTool()
results = ingredient_tool._run(user_id=102)
print(results)

[DEBUG] Tool invoked with user_id=102, pantry_table=None
[{'user_id': 102, 'pantry_item_id': 20004, 'pantry_name': 'kitchen_stock', 'product_name': 'rice', 'quantity': 2.0, 'used_quantity': 0, 'unit_of_measurement': 'kg', 'expiration_date': datetime.date(2025, 8, 10), 'created_at': datetime.datetime(2025, 5, 17, 9, 35, 40, 478671)}, {'user_id': 102, 'pantry_item_id': 20001, 'pantry_name': 'kitchen_stock', 'product_name': 'milk', 'quantity': 1.0, 'used_quantity': 0, 'unit_of_measurement': 'liter', 'expiration_date': datetime.date(2025, 8, 15), 'created_at': datetime.datetime(2025, 5, 17, 9, 35, 40, 478671)}, {'user_id': 102, 'pantry_item_id': 20005, 'pantry_name': 'kitchen_stock', 'product_name': 'vinegar', 'quantity': 6.0, 'used_quantity': 1, 'unit_of_measurement': 'ml', 'expiration_date': datetime.date(2025, 10, 1), 'created_at': datetime.datetime(2025, 5, 17, 9, 35, 40, 478671)}, {'user_id': 102, 'pantry_item_id': 20002, 'pantry_name': 'kitchen_stock', 'product_name': 'eggs', 'quanti

In [35]:
class UserRestrictionToolInput(BaseModel):
    user_id: int

class UserRestrictionTool(BaseTool):
    name: str = "UserPreferenceTool"
    description: str = "Fetches dietary restrictions and allergens for a user"
    args_schema: type[BaseModel] = UserPreferenceToolInput

    def _run(self, user_id: int) -> dict:
        query = """
            SELECT dietary_preference, allergens
            FROM `adsp-34002-on02-prep-sense.Inventory.user_preference`
            WHERE user_id = @user_id
        """

        job_config = bigquery.QueryJobConfig(
            query_parameters=[
                bigquery.ScalarQueryParameter("user_id", "INT64", user_id)
            ]
        )

        try:
            query_job = bq_client.query(query, job_config=job_config)
            result = list(query_job.result())
            if result:
                return dict(result[0])
            else:
                return {"message": "No dietary data found for this user."}
        except Exception as e:
            return {"error": str(e)}

In [36]:
user_restriction_tool = UserRestrictionTool()
results = user_restriction_tool._run(user_id=102)
print(results)

{'dietary_preference': ['vegetarian', 'low-carb'], 'allergens': ['gluten']}


In [29]:
# Define AI Agents for Pantry-to-Dinner Workflow
pantry_scan_agent = Agent(
    role="Pantry Scan Agent",
    goal="Retrieve list of available ingredients for user ID {user_id} from BigQuery pantry table",
    tools=[bigquery_tool],
    verbose=True,
    backstory="You have access to structured data sources like BigQuery and can extract relevant pantry contents."
)

ingredient_filter_agent = Agent(
    role="Ingredient Filter Agent",
    goal="Filter out expired or unusable items from the pantry list",
    tools=[ingredient_tool],
    verbose=True,
    backstory="You ensure only safe and usable ingredients are passed to recipe generation agents."
)

recipe_search_agent = Agent(
    role="Recipe Search Agent",
    goal="Find recipes that can be made using the filtered ingredients",
    tools=[search_tool, scrape_tool, ingredient_tool],
    verbose=True,
    backstory="You search online and through structured recipe repositories to find viable dinner options."
)

nutritional_agent = Agent(
    role="Nutritional Agent",
    goal="Evaluate the nutritional balance of each proposed recipe",
    tools=[search_tool],
    verbose=True,
    backstory="You ensure that dinner suggestions are healthy and balanced. Give me a detailed nutritional breakdown."
)

user_preferences_agent = Agent(
    role="User Preferences Agent",
    goal="Filter out ingredients and recipes that conflict with the user's dietary restrictions and allergens",
    tools=[UserRestrictionTool],
    verbose=True,
    backstory="You ensure the user only sees recipes and ingredients that comply with their health and dietary requirements."
)

recipe_scoring_agent = Agent(
    role="Recipe Scoring Agent",
    goal="Score recipes based on match with ingredients, nutrition, and user preferences",
    tools=[],
    verbose=True,
    backstory="You rank suggestions to make it easier for users to choose."
)

response_formatting_agent = Agent(
    role="Response Formatting Agent",
    goal="Format and summarize the best recipe suggestions for user-friendly viewing",
    tools=[],
    verbose=True,
    backstory="You turn results into clean summaries, including ingredients and nutrition info."
)

In [30]:
# Define Tasks for Agents

def create_task(agent, description, expected_output, input_variables=None):
    print(f"[DEBUG] Creating task for {agent.role} with inputs: {input_variables}")
    return Task(
        description=description,
        expected_output=expected_output,
        human_input=False,
        agent=agent,
        input_variables=input_variables or []
    )

In [37]:
tasks = [
    create_task(
        pantry_scan_agent,
        "Query BigQuery table for usable ingredients for a given user_id",
        "List of pantry ingredients for a given user_id",
        input_variables=["user_id", "pantry_table"]
    ),
        create_task(
        ingredient_filter_agent,
        "Filter out expired or unusable items from the user's pantry list based on expiration date and usage",
        "List of usable (non-expired) pantry ingredients for a given user_id",
        input_variables=["user_id", "pantry_table"]
    ),
    create_task(recipe_search_agent, "Find dinner recipes that use these ingredients", "List of at least 5 suitable recipes"),
    create_task(nutritional_agent, "Evaluate each recipe for nutritional value", "Nutritional score and comments for each recipe"),
    create_task(
        user_preferences_agent,
        "Filter recipes based on user dietary preferences",
        "List of safe ingredients or recipes based on user dietary restrictions",
        input_variables=["user_id"]
    ),
    create_task(recipe_scoring_agent, "Score recipes on match, nutrition, and user preference", "Ranked list of recipes"),
    create_task(response_formatting_agent, "Format top 3 recipes with ingredients, instructions, and nutrition summary", "Formatted response in markdown")
]

[DEBUG] Creating task for Pantry Scan Agent with inputs: ['user_id', 'pantry_table']
[DEBUG] Creating task for Ingredient Filter Agent with inputs: ['user_id', 'pantry_table']
[DEBUG] Creating task for Recipe Search Agent with inputs: None
[DEBUG] Creating task for Nutritional Agent with inputs: None
[DEBUG] Creating task for User Preferences Agent with inputs: ['user_id']
[DEBUG] Creating task for Recipe Scoring Agent with inputs: None
[DEBUG] Creating task for Response Formatting Agent with inputs: None


In [38]:
# Create and run the Crew
pantry_dinner_crew = Crew(
    agents=[
        pantry_scan_agent,
        ingredient_filter_agent,
        recipe_search_agent,
        nutritional_agent,
        user_preferences_agent,
        recipe_scoring_agent,
        response_formatting_agent
    ],
    tasks=tasks,
    verbose=True
)

inputs = {
 #   "user_dietary_preferences": "vegetarian, no nuts",
    "pantry_table": "adsp-34002-on02-prep-sense.Inventory.pantry_items",
    "user_id":102
}

In [39]:
print("[DEBUG] Final Inputs to Crew:", inputs)
result = pantry_dinner_crew.kickoff(inputs=inputs)
print(result)

[DEBUG] Final Inputs to Crew: {'pantry_table': 'adsp-34002-on02-prep-sense.Inventory.pantry_items', 'user_id': 102}


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: cb286e25-8dbc-4808-b7a8-83fb1353a8c5                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: cfcacfc3-d06a-442e-8225-4061d311696b
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: cfcacfc3-d06a-442e-8225-4061d311696b
       Status: Executing Task...
    └── 🤖 Agent: Pantry Scan Agent
            Status: In Progress

# Agent: Pantry Scan Agent
## Task: Query BigQuery table for usable ingredients for a given user_id


...

🚀 Crew: crew
├── 📋 Task: cfcacfc3-d06a-442e-8225-4061d311696b
│      Assigned to: Pantry Scan Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Pantry Scan Agent
│           Status: ✅ Completed
├── 📋 Task: d97d4cd7-dba9-41cb-bf86-b6de4e8cb036
│      Assigned to: Ingredient Filter Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Ingredient Filter Agent
│           Status: ✅ Completed
├── 📋 Task: 7e421f37-dc40-45ba-b0d0-0d57f7ee9483
│      Assigned to: Recipe Search Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Recipe Search Agent
│           Status: ✅ Completed
├── 📋 Task: 2499b46a-74de-4ec1-b1a3-25f4ddb8bb6c
│      Assigned to: Nutritional Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Nutritional Agent
│           Status: ✅ Completed
├── 📋 Task: e3b395d8-b91f-4d00-986a-7a84c2d21cc2
│      Assigned to: User Preferences Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: User Preferences Agent
│           Status: ✅ Completed
├── 📋 Task: 7504ed3f-6247-4635-bed4-11c8c09748cc
│      Assigned to: Recipe Scoring Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Recipe Scoring Agent
│           Status: ✅ Completed
└── 📋 Task: f8218216-8fea-42a2-9d91-c3d5e93c1ded
       Status: Executing Task...
    └── 🤖 Agent: Response Formatting Agent
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: cfcacfc3-d06a-442e-8225-4061d311696b
│      Assigned to: Pantry Scan Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Pantry Scan Agent
│           Status: ✅ Completed
├── 📋 Task: d97d4cd7-dba9-41cb-bf86-b6de4e8cb036
│      Assigned to: Ingredient Filter Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Ingredient Filter Agent
│           Status: ✅ Completed
├── 📋 Task: 7e421f37-dc40-45ba-b0d0-0d57f7ee9483
│      Assigned to: Recipe Search Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Recipe Search Agent
│           Status: ✅ Completed
├── 📋 Task: 2499b46a-74de-4ec1-b1a3-25f4ddb8bb6c
│      Assigned to: Nutritional Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Nutritional Agent
│           Status: ✅ Completed
├── 📋 Task: e3b395d8-b91f-4d00-986a-7a84c2d21cc2
│      Assigned to: User Preferences Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: User Preferences Agent
│           Status: ✅ Completed
├── 📋 Task: 7504ed3f-6247-4635-bed4-11c8c09748cc
│      Assigned to: Recipe Scoring Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Recipe Scoring Agent
│           Status: ✅ Completed
└── 📋 Task: f8218216-8fea-42a2-9d91-c3d5e93c1ded
       Assigned to: Response Formatting Agent
       Status: ✅ Completed
    └── 🤖 Agent: Response Formatting Agent
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: f8218216-8fea-42a2-9d91-c3d5e93c1ded                                                                     │
│  Agent: Response Formatting Agent                                                                               │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: cb286e25-8dbc-4808-b7a8-83fb1353a8c5                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Based on the user's available pantry ingredients, including rice, milk, vinegar, eggs, and sugar, I have prepared a selection of recipes that are suitable and utilize these resources effectively. Below are the top 3 recipe suggestions with detailed content for user-friendly viewing.

### 1. Breakfast Rice from Japan Recipe
**Description**: This simple yet flavorful recipe combines leftover rice, scrambled eggs, and soy sauce creating a nutritious and adaptable meal.

- **Ingredients**:
  - Leftover rice
  - 2 eggs
  - Soy sauce
  - Optional: Vegetables of choice (e.g., green onions, bell peppers)

- **Instructions**:
  1. Heat a pan and add rice until it's warm.
  2. Whisk eggs with a splash of soy sauce and pour into the pan.
  3. Stir gently until eggs are cooked through.
  4. Add soy sauce to taste and optionally toss in vegetables.

- **Nutrition Summary**:
  - Protein and carbohydrate-rich
  - Low in fat, especially if opting for non-stick cooking
  - Nutritional value can be incr

In [18]:
# Helper functions

def get_user_pantry_items(user_id: int):
    query = f"""
        SELECT *
        FROM `adsp-34002-on02-prep-sense.Inventory.pantry_items`
        WHERE pantry_id = {user_id} AND quantity > 0
    """
    try:
        query_job = bq_client.query(query)
        results = query_job.result()
        return [dict(row) for row in results]
    except Exception as e:
        return [{"error": str(e)}]

def insert_recipe(recipe_data: dict):
    table_id = "adsp-34002-on02-prep-sense.Inventory.recipies"
    errors = bq_client.insert_rows_json(table_id, [recipe_data])
    if errors == []:
        print("✅ New recipe added.")
    else:
        print("❌ Errors:", errors)